<a href="https://colab.research.google.com/github/sherrylittlefish/DLGAN/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**資料A**

In [2]:
import zipfile
import os

# 設定 zip 檔案路徑（根據你的實際位置修改）
zip_path = '/content/drive/MyDrive/高雄大學/pic.zip'
unzip_dir = '/content/unzipped'

# 解壓 pic.zip 到 /content/unzipped
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)


In [3]:
# 解壓 filter1.zip
filtered1_zip_path = os.path.join(unzip_dir, 'filtered1.zip')
filtered1_dir = os.path.join(unzip_dir, 'filtered1')
with zipfile.ZipFile(filtered1_zip_path, 'r') as zip_ref:
    zip_ref.extractall(filtered1_dir)

In [4]:
import shutil
import random

# 指定資料夾與輸出位置
folders = ['0051', '0052', '0053']
trainA_dir = '/content/dataset/trainA'
testA_dir = '/content/dataset/testA'

os.makedirs(trainA_dir, exist_ok=True)
os.makedirs(testA_dir, exist_ok=True)

# 收集所有圖片檔案路徑
all_images = []
for folder in folders:
    folder_path = os.path.join(filtered1_dir, folder)
    if os.path.isdir(folder_path):
        for fname in os.listdir(folder_path):
            fpath = os.path.join(folder_path, fname)
            if os.path.isfile(fpath):
                all_images.append(fpath)

# 打亂順序
random.shuffle(all_images)

# 分割資料集（約 1:10 比例）
split_index = max(1, int(len(all_images) * 0.1))  # 至少1張到testA
test_images = all_images[:split_index]
train_images = all_images[split_index:]

# 複製圖片
def copy_images(image_list, target_dir):
    for src_path in image_list:
        fname = os.path.basename(src_path)
        dst_path = os.path.join(target_dir, fname)
        # 若重名可加上 prefix 防止覆蓋（這裡為保簡單略過）
        shutil.copy(src_path, dst_path)

copy_images(train_images, trainA_dir)
copy_images(test_images, testA_dir)

print(f"✔ 總共 {len(all_images)} 張圖片，已分成：")
print(f"   ➤ trainA：{len(train_images)} 張")
print(f"   ➤ testA ：{len(test_images)} 張")


✔ 總共 1359 張圖片，已分成：
   ➤ trainA：1224 張
   ➤ testA ：135 張


**資料B**

In [5]:
zip_path2 = '/content/drive/MyDrive/高雄大學/person_images_filtered.zip'
# 解壓 person_images_filtered.zip 到 /content/unzipped
with zipfile.ZipFile(zip_path2, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

In [6]:
# 原始圖片資料夾
source_dir = '/content/unzipped/person_images_filtered'

# 輸出目錄
trainB_dir = '/content/dataset/trainB'
testB_dir = '/content/dataset/testB'

# 建立資料夾
os.makedirs(trainB_dir, exist_ok=True)
os.makedirs(testB_dir, exist_ok=True)


# 取得所有圖片清單
all_images = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]

# 打亂順序
random.shuffle(all_images)

# # 設定比例（約10%進 testB，其餘進 trainB）
# num_test = max(1, int(len(all_images) * 0.1))  # 至少 1 張
# test_images = all_images[:num_test]
# train_images = all_images[num_test:]

# 指定要切出來的數量
num_train = 1200
num_test = 150

# 確保不超過總數
num_train = min(num_train, len(all_images))
num_test = min(num_test, len(all_images) - num_train)

# 切分
train_images = all_images[:num_train]
test_images = all_images[num_train:num_train + num_test]

# 複製檔案
def copy_images(image_list, target_dir):
    for fname in image_list:
        src = os.path.join(source_dir, fname)
        dst = os.path.join(target_dir, fname)
        shutil.copy(src, dst)

copy_images(train_images, trainB_dir)
copy_images(test_images, testB_dir)

print(f"✔ 已完成分割：")
print(f"   ➤ trainB：{len(train_images)} 張")
print(f"   ➤ testB ：{len(test_images)} 張")


✔ 已完成分割：
   ➤ trainB：1200 張
   ➤ testB ：150 張


In [7]:
import torch
import torch.nn as nn

# 殘差區塊
class ResidualBlock(nn.Module):
    def __init__(self, dim):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(dim, dim, kernel_size=3),
            nn.InstanceNorm2d(dim),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(dim, dim, kernel_size=3),
            nn.InstanceNorm2d(dim),
        )

    def forward(self, x):
        return x + self.block(x)

# Generator 架構
class ResnetGenerator(nn.Module):
    def __init__(self, input_nc, output_nc, ngf=64, n_blocks=9):
        super(ResnetGenerator, self).__init__()

        model = [
            nn.ReflectionPad2d(3),
            nn.Conv2d(input_nc, ngf, kernel_size=7),
            nn.InstanceNorm2d(ngf),
            nn.ReLU(inplace=True),
        ]

        # Downsampling
        in_features = ngf
        out_features = in_features * 2
        for _ in range(2):
            model += [
                nn.Conv2d(in_features, out_features, kernel_size=3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features
            out_features *= 2

        # Residual Blocks
        for _ in range(n_blocks):
            model += [ResidualBlock(in_features)]

        # Upsampling
        out_features = in_features // 2
        for _ in range(2):
            model += [
                nn.ConvTranspose2d(in_features, out_features, kernel_size=3, stride=2, padding=1, output_padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True)
            ]
            in_features = out_features
            out_features = in_features // 2

        # Output layer
        model += [
            nn.ReflectionPad2d(3),
            nn.Conv2d(in_features, output_nc, kernel_size=7),
            nn.Tanh()
        ]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)


In [8]:
import torch.nn as nn

class PatchDiscriminator(nn.Module):
    def __init__(self, input_nc, ndf=64, n_layers=3):
        super(PatchDiscriminator, self).__init__()

        # 第一層：不使用 InstanceNorm
        layers = [
            nn.Conv2d(input_nc, ndf, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True)
        ]

        # 中間層
        nf_mult = 1
        for n in range(1, n_layers):
            nf_mult_prev = nf_mult
            nf_mult = min(2**n, 8)
            layers += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult,
                          kernel_size=4, stride=2, padding=1),
                nn.InstanceNorm2d(ndf * nf_mult),
                nn.LeakyReLU(0.2, inplace=True)
            ]

        # 最後一層（不 downsample）
        nf_mult_prev = nf_mult
        nf_mult = min(2**n_layers, 8)
        layers += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult,
                      kernel_size=4, stride=1, padding=1),
            nn.InstanceNorm2d(ndf * nf_mult),
            nn.LeakyReLU(0.2, inplace=True)
        ]

        # 輸出層：1 channel 的「真假 map」
        layers += [
            nn.Conv2d(ndf * nf_mult, 1, kernel_size=4, stride=1, padding=1)
        ]

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


In [9]:
import torch
import torch.nn as nn

# 判別器 loss 用 LSGAN
class GANLoss(nn.Module):
    def __init__(self):
        super(GANLoss, self).__init__()
        self.loss = nn.MSELoss()

    def get_target_tensor(self, prediction, target_is_real):
        if target_is_real:
            return torch.ones_like(prediction)
        else:
            return torch.zeros_like(prediction)

    def forward(self, prediction, target_is_real):
        target_tensor = self.get_target_tensor(prediction, target_is_real)
        return self.loss(prediction, target_tensor)


In [10]:
# 構造 loss function
criterion_GAN = GANLoss()
criterion_cycle = nn.L1Loss()
criterion_identity = nn.L1Loss()  # 可選


In [13]:
G_AB = ResnetGenerator(input_nc=3, output_nc=3)
G_BA = ResnetGenerator(input_nc=3, output_nc=3)
D_A = PatchDiscriminator(input_nc=3)
D_B = PatchDiscriminator(input_nc=3)

In [14]:
# 假設你已經定義：
# G_AB = A -> B Generator
# G_BA = B -> A Generator
# D_A = 判別 A 圖
# D_B = 判別 B 圖

# 優化器
import torch.optim as optim

lr = 0.0002
G_params = list(G_AB.parameters()) + list(G_BA.parameters())
optimizer_G = optim.Adam(G_params, lr=lr, betas=(0.5, 0.999))
optimizer_D_A = optim.Adam(D_A.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D_B = optim.Adam(D_B.parameters(), lr=lr, betas=(0.5, 0.999))


In [15]:
def train_step(real_A, real_B):
    ###### 1. 訓練 Generator ######
    optimizer_G.zero_grad()

    # A -> B
    fake_B = G_AB(real_A)
    pred_fake_B = D_B(fake_B)
    loss_GAN_AB = criterion_GAN(pred_fake_B, True)

    # B -> A
    fake_A = G_BA(real_B)
    pred_fake_A = D_A(fake_A)
    loss_GAN_BA = criterion_GAN(pred_fake_A, True)

    # Cycle consistency loss
    recovered_A = G_BA(fake_B)
    recovered_B = G_AB(fake_A)
    loss_cycle_A = criterion_cycle(recovered_A, real_A)
    loss_cycle_B = criterion_cycle(recovered_B, real_B)

    # Identity loss (optional)
    same_A = G_BA(real_A)
    same_B = G_AB(real_B)
    loss_idt_A = criterion_identity(same_A, real_A)
    loss_idt_B = criterion_identity(same_B, real_B)

    # 加總 loss
    lambda_cycle = 10.0
    lambda_id = 5.0
    loss_G = (
        loss_GAN_AB + loss_GAN_BA +
        lambda_cycle * (loss_cycle_A + loss_cycle_B) +
        lambda_id * (loss_idt_A + loss_idt_B)
    )

    loss_G.backward()
    optimizer_G.step()

    ###### 2. 訓練 Discriminator A ######
    optimizer_D_A.zero_grad()
    pred_real_A = D_A(real_A)
    pred_fake_A = D_A(fake_A.detach())
    loss_D_A = (
        criterion_GAN(pred_real_A, True) +
        criterion_GAN(pred_fake_A, False)
    ) * 0.5
    loss_D_A.backward()
    optimizer_D_A.step()

    ###### 3. 訓練 Discriminator B ######
    optimizer_D_B.zero_grad()
    pred_real_B = D_B(real_B)
    pred_fake_B = D_B(fake_B.detach())
    loss_D_B = (
        criterion_GAN(pred_real_B, True) +
        criterion_GAN(pred_fake_B, False)
    ) * 0.5
    loss_D_B.backward()
    optimizer_D_B.step()

    return {
        "loss_G": loss_G.item(),
        "loss_D_A": loss_D_A.item(),
        "loss_D_B": loss_D_B.item()
    }


In [ ]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

# 自訂 Dataset
class ImageDataset(Dataset):
    def __init__(self, root_A, root_B, transform=None):
        self.files_A = sorted([os.path.join(root_A, f) for f in os.listdir(root_A)])
        self.files_B = sorted([os.path.join(root_B, f) for f in os.listdir(root_B)])
        self.transform = transform

    def __len__(self):
        return min(len(self.files_A), len(self.files_B))

    def __getitem__(self, index):
        img_A = Image.open(self.files_A[index % len(self.files_A)]).convert('RGB')
        img_B = Image.open(self.files_B[index % len(self.files_B)]).convert('RGB')

        if self.transform:
            img_A = self.transform(img_A)
            img_B = self.transform(img_B)

        return {'A': img_A, 'B': img_B}

# 轉換設定（可微調）
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # [-1, 1]
])

# 路徑設定
train_dataset = ImageDataset('/content/dataset/trainA', '/content/dataset/trainB', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=2)


In [ ]:
import matplotlib.pyplot as plt

def denormalize(tensor):
    return tensor * 0.5 + 0.5  # 把 [-1,1] 還原到 [0,1]

def visualize_output(real_A, fake_B, real_B, fake_A):
    images = [real_A, fake_B, real_B, fake_A]
    titles = ['Real A', 'Fake B (A→B)', 'Real B', 'Fake A (B→A)']

    plt.figure(figsize=(10, 5))
    for i, img in enumerate(images):
        img = denormalize(img[0]).permute(1, 2, 0).cpu().detach().numpy()
        plt.subplot(1, 4, i+1)
        plt.imshow(img)
        plt.title(titles[i])
        plt.axis('off')
    plt.tight_layout()
    plt.show()


In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir='runs/cyclegan_experiment')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
G_AB.to(device)
G_BA.to(device)
D_A.to(device)
D_B.to(device)

# 訓練迴圈
n_epochs = 10

for epoch in range(1, n_epochs + 1):
    for i, data in enumerate(train_loader):
        real_A = data['A'].to(device)
        real_B = data['B'].to(device)

        losses = train_step(real_A, real_B)

        writer.add_scalar("Loss/G", losses['loss_G'], epoch * len(train_loader) + i)
        writer.add_scalar("Loss/D_A", losses['loss_D_A'], epoch * len(train_loader) + i)
        writer.add_scalar("Loss/D_B", losses['loss_D_B'], epoch * len(train_loader) + i)


        if i % 50 == 0:
            print(f"[Epoch {epoch}/{n_epochs}] [Batch {i}/{len(train_loader)}] "
                  f"Loss_G: {losses['loss_G']:.4f}, "
                  f"Loss_DA: {losses['loss_D_A']:.4f}, "
                  f"Loss_DB: {losses['loss_D_B']:.4f}")
        if i % 200 == 0:
            with torch.no_grad():
                fake_B = G_AB(real_A)
                fake_A = G_BA(real_B)
                visualize_output(real_A, fake_B, real_B, fake_A)
    # 每 N epoch 儲存一次
    if epoch % 2 == 0:
        torch.save(G_AB.state_dict(), f'G_AB_epoch{epoch}.pth')
        torch.save(G_BA.state_dict(), f'G_BA_epoch{epoch}.pth')
        torch.save(D_A.state_dict(), f'D_A_epoch{epoch}.pth')
        torch.save(D_B.state_dict(), f'D_B_epoch{epoch}.pth')
        print(f"✅ 已儲存模型於 epoch {epoch}")




In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs


In [ ]:
from torchvision.utils import save_image

def test_sample(model_G, input_dir, output_dir, transform):
    os.makedirs(output_dir, exist_ok=True)
    files = sorted(os.listdir(input_dir))
    model_G.eval()

    for fname in files:
        path = os.path.join(input_dir, fname)
        img = Image.open(path).convert('RGB')
        img_tensor = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            fake = model_G(img_tensor)

        out_img = denormalize(fake[0]).cpu()
        save_image(out_img, os.path.join(output_dir, fname))
    print(f"✅ 測試圖片儲存於：{output_dir}")


In [ ]:
test_sample(G_AB, '/content/dataset/testA', '/content/testA2B_output', transform)
test_sample(G_BA, '/content/dataset/testB', '/content/testB2A_output', transform)
